<a href="https://colab.research.google.com/github/semishen/Chinese-NLP-Practices/blob/main/05_network_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load data
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/finance_article.xlsx'
article = pd.read_excel(url)
print(article.shape)
article.head(10)

(300, 4)


,url,title,content,tag
0,https://www.ptt.cc/bbs/Loan/M.1586506724.A.914...,[問題] 新北原屋貸款,[抵押房屋資訊]\n地點：新北市永和\n房屋形態：透天五樓\n房屋種類：自有住宅\n房屋/土...,Loan
1,https://www.ptt.cc/bbs/Loan/M.1586449508.A.618...,信貸 (台積電),1. 信貸金額：80-200萬\n2. 還款期限：7年\n3. 貸款用途：投資理財\n4. ...,Loan
2,https://www.ptt.cc/bbs/Stock/M.1586573726.A.DB...,[新聞] 瑞幸咖啡遭遇做空 疫情危機下中概股受到的放大鏡式狙擊,1.原文連結：\n\n2.原文內容：\n林祖偉／BBC中文\n\n瑞幸咖啡有超過4500間分...,Stock
3,https://www.ptt.cc/bbs/Stock/M.1586582519.A.AE...,[新聞] 14檔業績四升 法人搶,1.原文連結：\n\n2.原文內容：14檔業績四升 法人搶\n2020-04-11 00:1...,Stock
4,https://www.ptt.cc/bbs/Stock/M.1586572758.A.8B...,[新聞] 突變酵素 塑膠瓶幾小時內能回收,1.原文連結：\n\n2.原文內容：\n科學家創造出一種突變細菌酵素，能在幾小時分解塑膠瓶進...,Stock
5,https://www.ptt.cc/bbs/creditcard/M.1586574045...,[討論] 大戶app卡頓造成轉帳功能被鎖,事情發生在禮拜五\n使用大戶專屬app轉帳\n接收OTP簡訊時發生卡頓\n10秒內連發10則...,creditcard
6,https://www.ptt.cc/bbs/creditcard/M.1586170596...,[閒聊] 一銀自動扣繳卡費100,2020.1.1-2020.6.30\n一銀有自動扣繳卡費折抵100元的活動\n應該算是長期...,creditcard
7,https://www.ptt.cc/bbs/Loan/M.1586605502.A.F24...,[問題] 個人信貸,代PO\n欲貸款金額: 30-50萬\n還款年限：5~7年\n貸款用途: 清償貸款，理財\n...,Loan
8,https://www.ptt.cc/bbs/Stock/M.1586613552.A.CB...,[標的] 大盤,1. 標的：大盤\n2. 分類：多/空/請益/心得:空/注意上方壓力與可能的再拉回\n3. ...,Stock
9,https://www.ptt.cc/bbs/Loan/M.1586397530.A.D19...,[問題] 信用貸款,欲貸款金額: 300萬。\n還款年限：7\n貸款用途: 投資理財\n工作性質: 科技業\n年...,Loan


In [ ]:
# jieba word segmentation
import jieba

corpus = []

for idx, row in article.iterrows():
  corpus.append(' '.join(jieba.cut(row['content'])))


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.836 seconds.
Prefix dict has been built successfully.


In [ ]:
# tf-idf vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
ti_idf_vectorizer = TfidfVectorizer()
tf_idf_matrix = ti_idf_vectorizer.fit_transform(corpus)
tf_idf_matrix

<300x9613 sparse matrix of type '<class 'numpy.float64'>'
	with 26429 stored elements in Compressed Sparse Row format>

In [ ]:
# cosine distance
from sklearn.metrics.pairwise import cosine_distances
cs = cosine_distances(tf_idf_matrix)

In [ ]:
# k-means clustering based on cosine distance as baseline
cs = 1.0 - cs # convert to cosine similarity

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=10)
results = kmeans.fit_predict(cs) 

In [ ]:
# inertia, silhouette_score
from sklearn.metrics import silhouette_score
sil_score = silhouette_score(cs, labels= results)
print('silhouette_score: ', sil_score)
print('kmeans_inertia: ', kmeans.inertia_)

silhouette_score:  0.3457876328483865
kmeans_inertia:  331.30214101662176


In [ ]:
# find N groups for best K-means performance
for i in range(2, 21):
  kmeans = KMeans(n_clusters=i, init='k-means++', random_state=10)
  results = kmeans.fit_predict(cs)
  sil_score = silhouette_score(cs, labels= results) 
  print(f'{i}: inertia: {kmeans.inertia_} | silhouette: {sil_score}')

2: inertia: 404.8902337537491 | silhouette: 0.4792899372760053
3: inertia: 348.04359834605896 | silhouette: 0.3998093028108795
4: inertia: 331.30214101662176 | silhouette: 0.3457876328483865
5: inertia: 316.9941384919731 | silhouette: 0.13121440911687174
6: inertia: 307.71989316289057 | silhouette: 0.1306757608088468
7: inertia: 300.2127197132053 | silhouette: 0.10076783837545505
8: inertia: 291.7910681002314 | silhouette: 0.10606297325411058
9: inertia: 287.10359281478105 | silhouette: 0.14394204120191786
10: inertia: 279.48159782712366 | silhouette: 0.11292496524002223
11: inertia: 273.9001966189624 | silhouette: 0.11918708047977884
12: inertia: 269.3531872793301 | silhouette: 0.11824721537253619
13: inertia: 265.8725891105207 | silhouette: 0.10554541149876352
14: inertia: 260.8197781679419 | silhouette: 0.1097095374537505
15: inertia: 258.83473541184503 | silhouette: 0.10809598727211181
16: inertia: 256.40543804534235 | silhouette: 0.10682634962947743
17: inertia: 253.5218115321036 

In [ ]:
# maybe clustering cs in 2-4 is appropriate, try 4
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=10)
results_labels = kmeans.fit_predict(cs)

# print clusters
for id in range(4):
  print(f'========={id+1}=========')
  for idx, c in enumerate(results_labels):
    if c == id:
      print(article['title'][idx])

=========1=========
[新聞] 瑞幸咖啡遭遇做空 疫情危機下中概股受到的放大鏡式狙擊
[新聞] 14檔業績四升 法人搶
[新聞] 突變酵素 塑膠瓶幾小時內能回收
[討論] 大戶app卡頓造成轉帳功能被鎖
[閒聊] 一銀自動扣繳卡費100
[標的] 大盤
[情報] 聯合信用卡公司將停用磁條卡
[新聞] 聽說有人會背信用卡卡號…「前面那麼長」
[問題] 刷完卡沒有簡訊通知，是正常的嗎？
[新聞] 中華開發金控公告董事會決議108年度股利
[新聞] 經濟被新冠害死！義嗆和歐盟「同歸於盡」
[新聞] 兆豐金、兆豐銀3月雙現虧損 銀行稅後慘賠
[心得] 台新GOGO卡 小白核卡
Re: [標的] 台指期 多
[討論] 商務鈦金卡與御璽卡屬於公司卡?
Re: [請益] 請問配股配息
[問題] 渣打現金回饋刷學費
[請益] 美股有qe救 台股有什麼?
[問題] 首購青年貸款疑問
[請益] 財報實例說明網站
[新聞] 聯儲局資產負債表規模首破6萬億美元
[問題] line pay money繳聯邦卡費問題
[請益] 無限QE跟囚徒理論誰會贏？
[心得] 學生小白 中信LP+秀泰/玉山Pi 核卡
[新聞] 一年內砍22張信用卡 玉山銀 「減法」下
Re: [討論]  沒辦法出國，各位會剪年費卡嗎？
[問題] 營業稅刷卡拆單問題
[新聞] 原油減產談判仍未定案　沙烏地阿拉伯推
[心得] 第二支腳傳說
[情報] VISA在台推動代碼化到商家及EMV 3-DS驗證
Re: [請益] 關於元大原油正2(00672L)追蹤標的
[問題] 永豐大戶無法綁定Netflix
[問題] 玉山pi ubear未核卡問題
Re: [新聞] 川普伸援手 美墨同意共同減產35萬桶原油
[新聞] 川普伸援手 美墨同意共同減產35萬桶原油
Re: [討論] 沒辦法出國，各位會剪年費卡嗎？
[新聞] 《業績-金融》統一證Q1每股虧0.673元
[其他] 台新金 109年3月合併自結盈餘
[標的] 9943 好樂迪 多
[問題] 請問大家都是用什麼方法提醒自己不被收年
[請益] 如果漲停已經達到你預設的停利點?
[心得] 學貸小資渣打現金回饋核卡
Re: [請益] 請問配股配息
Re: [情報] 兆豐送萬瓶乾洗手和口罩幫你付運費
[其他] 中壽109年3月自結損益
[問題] 徵求國泰人壽房貸業務
[

In [ ]:
# use DBSCAN

from sklearn.cluster import DBSCAN
results_labels = DBSCAN(eps=1.3).fit_predict(cs)
num_clusters = max(results_labels)

# print clusters
print(num_clusters)
for id in range(num_clusters):
  print(f'========={id+1}=========')
  for idx, c in enumerate(results_labels):
    if c == (id-1):
      print(article['title'][idx])

8
=========1=========
[新聞] 14檔業績四升 法人搶
[新聞] 突變酵素 塑膠瓶幾小時內能回收
[閒聊] 一銀自動扣繳卡費100
[標的] 大盤
[情報] 聯合信用卡公司將停用磁條卡
[新聞] 聽說有人會背信用卡卡號…「前面那麼長」
[問題] 刷完卡沒有簡訊通知，是正常的嗎？
[新聞] 中華開發金控公告董事會決議108年度股利
[新聞] 經濟被新冠害死！義嗆和歐盟「同歸於盡」
Re: [標的] 台指期 多
[問題] 渣打現金回饋刷學費
[請益] 美股有qe救 台股有什麼?
[問題] 首購青年貸款疑問
[請益] 財報實例說明網站
[新聞] 聯儲局資產負債表規模首破6萬億美元
[問題] line pay money繳聯邦卡費問題
[請益] 無限QE跟囚徒理論誰會贏？
Re: [討論]  沒辦法出國，各位會剪年費卡嗎？
[問題] 營業稅刷卡拆單問題
[心得] 第二支腳傳說
[情報] VISA在台推動代碼化到商家及EMV 3-DS驗證
[問題] 玉山pi ubear未核卡問題
Re: [新聞] 川普伸援手 美墨同意共同減產35萬桶原油
Re: [討論] 沒辦法出國，各位會剪年費卡嗎？
[標的] 9943 好樂迪 多
[問題] 請問大家都是用什麼方法提醒自己不被收年
[請益] 如果漲停已經達到你預設的停利點?
Re: [情報] 兆豐送萬瓶乾洗手和口罩幫你付運費
[問題] 徵求國泰人壽房貸業務
[問題] 軟體自動續約扣款(應該非月經文..QQ)
[新聞] 口罩實名制販賣機登場 微星AIoT技術展實
[問題] 台北萬華轉貸
[情報] 萬事達提高29國感應免簽上限
[問題] 想申辦 樂天卡 要怎麼辦
[請益] 無限QE只有美國能做到嗎？
[標的] 展旺後續討論
[問題] 永豐原卡需要停卡再申辦大戶卡嗎?
[閒聊] 秀泰聯名卡 平日5折 (憑簽單)
[討論]  沒辦法出國，各位會剪高卡嗎？
[問題] 2月份的新光三月卡利嗨
[心得] 美股現在是否處於不理性的繁榮?
[標的] 奇力新 空
[問題] 玉山pi卡 mastercard logo問題
[情報] Samsung pay悠遊卡儲值
[新聞]營收：雄獅(2731)3月營收3億8325萬元
[問題] 股票質押
[新聞] 刷卡網購安全系統 升級
[請益] 目前能維持萬點以上是不是都是政府護盤?
[新聞] 大陸債

In [ ]:
# network graph, community detection
import networkx as nx
import community

criterias = [0.061, 0.062, 0.063, 0.064, 0.065]

for ceiteria in criterias:
  nm = (cs > ceiteria).astype(int) # network matrix
  G = nx.from_numpy_matrix(nm)
  comm = community.best_partition(G)
  clusters = max(comm.values())
  print(f'ceiteria: {ceiteria} | clusters: {clusters}')

ceiteria: 0.061 | clusters: 3
ceiteria: 0.062 | clusters: 3
ceiteria: 0.063 | clusters: 4
ceiteria: 0.064 | clusters: 4
ceiteria: 0.065 | clusters: 4


In [ ]:
# choose 0.062 as the criteria
nm = (cs > 0.062).astype(int) # network matrix
G = nx.from_numpy_matrix(nm)
comm = community.best_partition(G)

# clustering
article_cluster = []
for id in range(3):
  temp_dic={}
  temp_num = 0
  temp_articles = []
  for idx, c in enumerate(comm.values()):
    if c == id:
      temp_num += 1
      temp_articles.append(article['title'][idx])
  temp_dic['cluster'] = id+1
  temp_dic['num_of_articles'] = temp_num
  temp_dic['articles'] = temp_articles
  article_cluster.append(temp_dic)

article_cluster

[{'articles': ['[問題] 新北原屋貸款',
   '信貸 (台積電)',
   '[問題] 個人信貸',
   '[問題] 信用貸款',
   '[問題] 竹北房貸(首購)',
   '[問題] 個人信貸',
   '[問題] 個人信貸',
   '[問題] 台南 房屋抵押貸款',
   '[問題] 個人信貸',
   '[問題] 個人信貸',
   '[問題] 彰化公教首購房貸',
   '[問題] 台中新光重劃區房屋轉貸',
   '[問題] 竹東房貸(首購)',
   '[問題] 個人信用貸款',
   '[問題] 房屋轉貸（新北）',
   '[問題] 徵求國泰人壽房貸業務',
   '[問題] 南投公教首購房貸',
   '[問題] 房屋貸款（三峽北大特區）',
   '[問題] 台北萬華轉貸',
   '[問題] 竹北房貸轉貸',
   '[問題] 個人信貸',
   '[問題] 個人信貸',
   '[問題] 個人信貸',
   '[問題] 台南市 外籍人士房貸',
   '[問題] 個人信貸',
   '[問題] 個人信貸',
   '[問題] 股票質押',
   '[問題] 房貸新北',
   '[問題] 北市醫師自住房貸',
   '[問題] 淡水紅樹林首購貸款',
   '[問題]  個人首次信貸',
   '[問題] 個人信貸',
   '[請益] 小額貸款',
   '[問題] 購買農地貸款',
   '[問題] 個人信貸',
   '個人信貸',
   '[問題] 個人信貸',
   '[問題] 個人信貸',
   '[問題] 建地貸款',
   '[問題] 個人信用貸款',
   '[問題] 補習班主任 信貸',
   '[問題] 台北房屋增貸',
   '[問題] 首次個人信貸',
   '[問題] 台中潭子首購房貸',
   '[問題] 個人信貸',
   '[問題] 新竹 金雅房貸(首購）',
   '台北市原屋貸款',
   '[問題] 醫師信貸',
   '[問題] 個人信貸',
   '[問題] 個人信貸',
   '[問題] 個人信貸',
   '[問題] 新竹 首購房貸',
   '[問題] 個人信貸',
   '[請益] 個人循環動用信貸',
   '[問題] 竹北房貸 第二房',
   '[問題] 頭